## Проект "Изучение базы данных приложения для чтения книг"

В данном проекте производится анализ базы данных, используемой в приложении для чтения книг.
Цель - сформулировать рекомендации по улучшению продукта.


##### Описание данных

**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

##### Настройка подключения

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine

In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

##### Чтение и вывод данных из исходных таблиц

In [3]:
query = ''' SELECT *
            FROM books
        '''

In [4]:
books = pd.io.sql.read_sql(query, con = engine)
books.head()

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [5]:
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   book_id           1000 non-null   int64 
 1   author_id         1000 non-null   int64 
 2   title             1000 non-null   object
 3   num_pages         1000 non-null   int64 
 4   publication_date  1000 non-null   object
 5   publisher_id      1000 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 47.0+ KB


In [6]:
query = ''' SELECT *
            FROM authors
        '''

In [7]:
authors = pd.io.sql.read_sql(query, con = engine)
authors.head()

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [8]:
query = ''' SELECT *
            FROM publishers
        '''

In [9]:
publishers = pd.io.sql.read_sql(query, con = engine)
publishers.head()

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [10]:
query = ''' SELECT *
            FROM ratings
        '''

In [11]:
ratings = pd.io.sql.read_sql(query, con = engine)
ratings.head()

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [12]:
query = ''' SELECT *
            FROM reviews
        '''

In [13]:
reviews = pd.io.sql.read_sql(query, con = engine)
reviews.head()

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


### Задача 1.

In [14]:
# сколько книг вышло после 2000-01-01
query = '''SELECT COUNT(book_id)
                FROM books
                WHERE publication_date > '2000-01-01'
                ;
                '''
pd.io.sql.read_sql(query, con = engine)

,count
0,819


После 01 января 2000 года вышло 819 книг.

### Задача 2.

In [15]:
query = '''
            SELECT
                books.title,
                (
                    SELECT
                        COUNT(review_id) AS review_cnt
                    FROM reviews
                    WHERE book_id = books.book_id
                ),
                (
                    SELECT
                        AVG(rating)
                    FROM ratings
                    WHERE book_id = books.book_id) AS avg_rating
            FROM books, reviews
            GROUP BY books.book_id
            ORDER BY review_cnt DESC
        '''
pd.io.sql.read_sql(query, con=engine)

,title,review_cnt,avg_rating
0,Twilight (Twilight #1),7,3.662500
1,Water for Elephants,6,3.977273
2,The Glass Castle,6,4.206897
3,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634
4,The Curious Incident of the Dog in the Night-Time,6,4.081081
...,...,...,...
995,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.666667
996,The Natural Way to Draw,0,3.000000
997,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.000000
998,Essential Tales and Poems,0,4.000000


Выведена информация о количестве рецензий и среднем рейтинге по каждой книге.

### Задача 3.

In [16]:
# издательство, которое выпустило наибольшее число книг толще 50 страниц 
query = '''SELECT pub.publisher,
COUNT(b.book_id) AS books_count
                FROM books AS b
                LEFT JOIN publishers AS pub ON b.publisher_id=pub.publisher_id
                WHERE b.num_pages > 50
                GROUP BY pub.publisher
                ORDER BY books_count DESC
                LIMIT 1;
                '''
pd.io.sql.read_sql(query, con = engine)

,publisher,books_count
0,Penguin Books,42


Наибольшее число книг толщиной более 50 страниц - 42,  выпустило издательство Penguin Books.

### Задача 4.

In [17]:
# автор с самой высокой средней оценкой книг — только книги с 50 и более оценками
query = '''WITH r AS (SELECT AVG(rating) AS av_r, book_id
                      FROM ratings
                      GROUP BY book_id
                      HAVING COUNT(rating_id) >= 50)
         SELECT a.author, AVG(r.av_r)
         FROM books AS b JOIN r ON b.book_id=r.book_id
         JOIN authors AS a ON b.author_id=a.author_id
         GROUP BY a.author
         ORDER BY AVG(r.av_r) DESC
         LIMIT 1;
                '''
pd.io.sql.read_sql(query, con = engine)

,author,avg
0,J.K. Rowling/Mary GrandPré,4.283844


Автор с самой высокой средней оценкой книг - 4.3, которому пользователями выставлено 50 и более оценок - J.K. Rowling/Mary GrandPré.

### Задача 5.

In [18]:
# среднее количество обзоров от пользователей, которые поставили больше 50 оценок
query = '''WITH rat_50 AS (SELECT username
                             FROM ratings
                             GROUP BY username
                             HAVING COUNT(rating_id) > 50),
                rev_count AS (SELECT COUNT(review_id) AS count,
                username
                FROM reviews
                GROUP BY username)                     
SELECT ROUND (AVG(rev_count.count)) AS avg_counts_reviews
FROM rat_50 INNER JOIN rev_count ON rat_50.username = rev_count.username;'''

pd.io.sql.read_sql(query, con = engine)

,avg_counts_reviews
0,24.0


Среднее количество обзоров от пользователей, которые одновременно поставили больше 50 оценок, - 24.

### Вывод
- Добавить на стартовой странице приложения книги с высоким рейтингом,
- Вывести на главный экран рекомендуемого автора,
- Ввести систему поощрений пользователей за написанные обзоры и поставленные оценки,
- Промоакции по книгам издательства Penguin Books.